In [0]:
import numpy as np
import pandas as pd
import datetime as pd
server = '172.16.1.38'
username = 'sa'
password = 'qwerty123'
driver = '{ODBC Driver 17 for SQL Server}'
cn = pyodbc.connect('DRIVER=' + driver + ';SERVER=' + server + ';UID=' + username + ';PWD=' + password)

Primero creamos el objeto miCartera el cuál guardará toda la información necesaria. Para crearla debemos pasarle arreglos que guarden los nemotecnicos de los bonos, los ID_Keys de los derivados y el nombre del archivo de las acciones. Todo lo anterior como string. Además, se entrega la fecha de valorización y la conexión a base de datos. La información de los instrumentos se busca en:

* Nemotecnicos (bonos):  TdNemoRF
* ID_Keys (derivados): TdCarteraDerivados_V2
* Acciones: Yahoo finance.




In [0]:
server = '172.16.1.38'
username = 'sa'
password = 'qwerty123'
driver = '{ODBC Driver 17 for SQL Server}'
cn = pyodbc.connect('DRIVER=' + driver + ';SERVER=' + server + ';UID=' + username + ';PWD=' + password)
miCartera = Cartera(datetime.date(2018, 4, 18), ["BSTDU10618"], ["146854"], ["BSANTANDER.SN.xlsx"], cn)


Al momento de crear la cartera se obtienen los riesgos y monedas de los bonos y las monedas de los derivados tanto para sus activos y pasivos. Con la información anterior se tienen los siguientes parámetros:

* $n_a$ la cantidad de acciones.
* $n_d$ la cantidad de derivados (separados por activo y pasivo (?)).
* $n_b$ la cantidad de bonos.
* $r$ la cantidad de riesgos.
* $m$ la cantidad de monedas.

Como los bonos y derivados dependen de curvas de factores de descuento (?), el análisis de estos se centra en como evoluciona temporalmente este factor para plazos fijos. Los plazos, que se miden en años, son:

$Plazos=\left\{   0.083, 0.25, 0.5, 1, 2, 3, 4, 5, 7, 9, 10, 15, 20, 30  \right\}$

Para obtener el histórico de factores de descuento asociado a un bono, se necesita su moneda y riesgo con lo cuál se pueden buscar las curvas en:

* TdCurvaNS
* TdCurvasSector

Para obtener el histórico de factores de descuento asociado al activo o pasivo de un derivado, se necesita su moneda con lo cuál se pueden buscar las curvas en:

* TdCurvaDerivados


A partir de este histórico, denotado por $(p_d)_{d\in {1,...,T}}$ con $T$ el día más reciente, se calcula el retorno de la siguiente forma:

$r_i = ln\left( \frac{p_i}{p_{i-1}}\right) \ i>2$

$r_1 = 0$  (Sólo para que las dimensiones sigan calzando)

El caso de las acciones es más simple puesto que contamos con un histórico de precios de estas, lo cuál facilita el calculo de sus retornos. Una vez teniendo los retornos de las acciones y de los plazos para cada moneda en el caso de los derivados y para cada moneda y riesgo en el caso de los bonos, se pasa a calcular covarianzas y volatilidades. La covarianza entre los retornos $i$ y $j$ se define como:

$ \sigma_{i,j} = \frac{1-\lambda}{1-\lambda^{T-1}}\sum_{d = 1}^{T}{\lambda^{d-1} r_{i,T-d +1} r_{j,T-d +1} }$

Por otra parte la volatilidad del retorno $i$ será $\sigma_i = \sqrt{\sigma_{i,i}}$. Las correlaciones se definirán como:

$\rho_{i,j} = \frac{\sigma_{i,j}}{\sigma_i \sigma_j}$

En las definiciones anteriores se usó $\textit{EWMA}$ puesto que se busca que los datos más recientes tengan un mayor peso en la estimación de los parámetros del modelo.

La clase cartera cuenta con un método $\textit{get_retAcciones}$ el cuál entregará un $\textit{DataFrame}$ con los retornos de acciones y análogo tiene el método $\textit{get_retCurvas}$. El último método tiene ese nombre puesto que procesa los instrumentos cuyo análisis se hace buscando curvas en la base de datos. A continuación unos ejemplos:



In [0]:
# Print de DataFrames de retornos. Notar que el orden en la tabla es acciones-plazosDerivados-plazosBonos.
miCartera.get_retAcciones
miCartera.get_retCurvas(100) # Se le entrega la cantidad de curvas deseadas.

Cabe mencionar que al momento de crear el objeto miCartera, se realiza todo el proceso de calculo de correlaciones y volatilidades. En la clase existen los métodos $\textit{get_volAcciones}$ y $\textit{get_volCurvas}$ encargados de retornar volatilidades para cada grupo de instrumentos. 

In [0]:
# Print de DataFrames con volatilidades.
miCartera.get_volAcciones()
miCartera.get_volCurvas()

Después se pasa a unir la información que se obtiene por separado. Para esto se usan los métodos $\textit{get_volatilidades()}$ y $\textit{retornos_totales}$. Luego, a partir de los retornos se puede calcular la matríz de correlaciones con $\textit{correlaciones_totales}$.

In [0]:
# Print de DataFrame de la volatilidad y retornos de todos los intrumentos.
miCartera.get_volatilidades()
miCartera.retornos_totales()
# Print de matríz de correlaciones.
miCartera.correlaciones_totales() 

Una vez obtenida la matríz de correlaciones y el vector de volatilidades, se procede a $\textit{pivotear}$ los pagos venideros de los intrumentos que se analizan a partir de curvas (Bonos y Derivados). Para la realización de este método se siguió el documento $\textit{Metodología cálculo de indicadores VaR}$. La clase tiene los métodos $\textit{get_pivotes_bonos}$ y $\textit{get_pivotes_derivados}$ que retornan esta información.

In [0]:
# Print del resultado de pivotear derivados y bonos.
miCartera.get_pivotes_bonos()
miCartera.get_pivotes_derivados()

Para finalmente calcular la volatilidad de la cartera $C$ en su completitud, se define el vector de valores presentes de la cartera como el vector que guarda el valor presente de las acciones, los plazos en cada moneda (derivados) y de plazos en cada moneda y riesgo (bonos). Este vector se denominará como $V_C \in \mathbb{R}^N$ con $N = n_a + 14n_dm + 14n_bmr$.

Se modela el retorno de la cartera como $R=\sum_{i\in I}{w_iR_i}$ con $I$ el conjunto de los instrumentos y $w_i$ el peso del instrumento con respecto a la cartera.